In [1]:
%load_ext autoreload

In [2]:
%autoreload now

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
os.environ["WANDB_API_KEY"] = ""

# Download and convert data

In [4]:
from document_segmentation.pagexml.annotations.renate_analysis import (
    RenateAnalysis,
    RenateAnalysisInv,
)
from document_segmentation.pagexml.annotations.sheet import Sheet
from document_segmentation.settings import RENATE_ANALYSIS_DIR, RENATE_ANALYSIS_SHEETS

sheets: list[Sheet] = [RenateAnalysis()] + [
    RenateAnalysisInv(sheet) for sheet in RENATE_ANALYSIS_SHEETS
]

In [5]:
N = None

for sheet in sheets:
    sheet.download(RENATE_ANALYSIS_DIR, N)

Writing documents: 0doc [00:00, ?doc/s]
Writing documents: 100%|██████████| 26/26 [00:11<00:00,  2.36doc/s]
Writing documents: 184doc [00:25,  7.32doc/s]                      


# Load Data

In [6]:
TRAINING_DATA = 0.8

In [7]:
from document_segmentation.model.dataset import DocumentDataset

dataset: DocumentDataset = DocumentDataset.from_dir(RENATE_ANALYSIS_DIR)
dataset.shuffle()

len(dataset)

Reading JSON files: 100%|██████████| 285/285 [00:01<00:00, 183.94file/s]


285

In [8]:
dataset._class_counts()

Counter({<Label.IN: 1>: 3206,
         <Label.BEGIN: 0>: 285,
         <Label.END: 2>: 281,
         <Label.OUT: 3>: 170})

In [9]:
dataset.class_weights()

[0.9965034965034965,
 0.08886810102899906,
 1.0106382978723405,
 1.6666666666666667]

In [10]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [11]:
training_data._class_counts()

Counter({<Label.IN: 1>: 2648,
         <Label.BEGIN: 0>: 228,
         <Label.END: 2>: 226,
         <Label.OUT: 3>: 113})

In [12]:
test_data._class_counts()

Counter({<Label.IN: 1>: 558,
         <Label.BEGIN: 0>: 57,
         <Label.OUT: 3>: 57,
         <Label.END: 2>: 55})

# Train Model

In [13]:
BATCH_SIZE = 128
EPOCHS = 10
WEIGHTS = dataset.class_weights()

In [14]:
%autoreload now

In [15]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [16]:
tagger._device

'mps'

In [17]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_feature

In [18]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: carschno. Use `wandb login --relogin` to force relogin


Training:  19%|█▉        | 43/229 [00:28<01:54,  1.62batch/s]

In [ ]:
import torch

with open("page_sequence_tagger.pt", "xb") as f:
    torch.save(tagger, f)

FileExistsError: [Errno 17] File exists: 'page_sequence_tagger.pt'

# Evaluate Model

In [ ]:
import sys

precision, recall, f1, accuracy = tagger.eval_(test_data, BATCH_SIZE, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:   4%|▎         | 2/57 [00:00<00:19,  2.88batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1631_0289.jpg	Twee Copie translaat briefien; door de hofs-groten	[6.53625278346226e-08, 5.507481546374038e-07, 6.621453962907253e-08, 0.9999992847442627]
OUT	IN	NL-HaNA_1.04.02_1631_0290.jpg	wederseijdse gesonth:t &:a bestaande :/ gevoerd; z	[1.3031361767446015e-08, 1.1301094815507895e-07, 1.000465044143084e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1631_0291.jpg	Luijden sig onthielden, en ondervondt; dat die dri	[1.035660535109173e-08, 8.578255261681988e-08, 7.434924231830564e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1631_0292.jpg	daarom niet en konden gaan, derhalve; gelieve uEd:	[9.09468589327389e-09, 7.797535772624542e-08, 7.22600868030554e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1631_0293.jpg	als 't haar maar In de zin quam; baldadigh, en sto	[1.006208094622707e-08, 7.44559969234615e-08, 7.060126705482617e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1631_0294.jpg	hebben verstaan, en begrepen: waar—; na nogh eenig	[9.530530142853877e-0

Evaluating:   5%|▌         | 3/57 [00:01<00:23,  2.34batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1074_0385.jpg	General wegens de ho. Hoog. Mogende heeren staten;	[6.229766569276762e-08, 6.046818725735648e-07, 7.96727945839848e-08, 0.9999992847442627]
OUT	IN	NL-HaNA_1.04.02_1074_0386.jpg	volgen de hoofden vant eylant oma; Don George hoof	[1.1975734182101405e-08, 1.2049828512772365e-07, 1.060981258405036e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1074_0387.jpg	Gemerckt onse onderdaennen alreede hier langentyt 	[9.848228010866933e-09, 8.8973749257093e-08, 7.803199864042654e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1074_0388.jpg	Volgen de negrijs staende onder rosseniue; rosseni	[9.587836302671349e-09, 7.876820973251597e-08, 7.3524542010261484e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1074_0389.jpg	Orangeaya larricca, lamme en baro hebben vangelyck	[9.263525946323625e-09, 8.726132705305645e-08, 7.16070092110499e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1074_0390.jpg	Casteel souden laten vinden onder beloften dat hy;	[9.98608129521017e-0

...

OUT	BEGIN	NL-HaNA_1.04.02_1557_0667.jpg	Van Malacca onder deno 1:e' Februarij ao 194.; Toe	[6.228722071455195e-08, 6.030359713804501e-07, 7.951808811412775e-08, 0.9999992847442627]
OUT	IN	NL-HaNA_1.04.02_1557_0668.jpg	Van Malacca onder Dato 11=en Februarij A:o 1694.; 	[1.192836229790828e-08, 1.2104675306545687e-07, 1.0667175587286692e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0669.jpg	Van Malacca onder dato 11:e' Februarij ao 1694; la	[9.810731782522453e-09, 9.123461097715335e-08, 7.82584130831765e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0670.jpg	Van Malacca onder dato 11=e' Februarij A=o 1694.; 	[9.518504207051137e-09, 7.969882886982305e-08, 7.419695080557176e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0671.jpg	Van Malaca onder dato 15:en Februarij Ao 194; port	[9.155515456882313e-09, 8.837316300969178e-08, 7.265008150625363e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0672.jpg	Van Malaca onder dato 11„en Februarij A:o 1644:; h	[9.956184321424644

Evaluating:  98%|█████████▊| 56/57 [00:25<00:00,  2.77batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1557_0909.jpg	Van Sumatras W=t Cust onder dato 25:e Januarij A=o	[5.198805297368381e-08, 5.882599793949339e-07, 7.535977175621156e-08, 0.9999992847442627]
OUT	IN	NL-HaNA_1.04.02_1557_0910.jpg	Van Sumatras W:t Cust onder dato 25:en Ianuarij A:	[1.2555656958568306e-08, 1.1129431953804669e-07, 1.1284503109720845e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0911.jpg	Van Sumatras W=t Cust onder dato 25:' Januarij a:o	[9.864979055862477e-09, 8.497742953750276e-08, 8.143410390459849e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0912.jpg	Van Sumatras W„t Cust onder dato 25:en Januarij A:	[8.896758885157396e-09, 8.020480635195781e-08, 7.407223723276957e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0913.jpg	Van Sumatras W„t Cust onder dato 25:en Januarij A:	[9.782031185068263e-09, 7.712912264423721e-08, 7.3296595459737546e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1557_0914.jpg	Van Sumatras W„tCust onder dato 25=en Januarij A:o	[9.506058162855

Evaluating: 100%|██████████| 57/57 [00:26<00:00,  2.12batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1083_0013.jpg	adn. 6=en augustij Anoo 1624; Octe der kerckelijck	[6.32070680239849e-08, 5.478199227582081e-07, 7.218459074920247e-08, 0.9999992847442627]
OUT	IN	NL-HaNA_1.04.02_1083_0014.jpg		[1.2907528379457744e-08, 1.247105529955661e-07, 1.080869704850329e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1083_0015.jpg	Int Jaer des Heeren onses Salichmaeckers Jesu Chri	[1.02633368470606e-08, 8.646376414844781e-08, 7.83771625378904e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1083_0016.jpg	Kercken dienaers soo te lande als opde; Rhede alhi	[9.152529401035281e-09, 8.366508552626328e-08, 7.754431763373759e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1083_0017.jpg	Verlost moeten werden midtsgaders offf oock eenige	[8.738303414190796e-09, 8.684506269673875e-08, 7.606608676269389e-09, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1083_0018.jpg	Alsoo de Generale Comp. e. dageluckx boven haer ve	[8.759295511140408e-09, 9.568967840323239e-08, 8.007896568074102e-09, 0

In [ ]:
import csv
import sys

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{metric.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	BEGIN	IN	END	OUT
MulticlassPrecision	0.0000	0.0000	0.0000	0.0348
MulticlassRecall	0.0000	0.0000	0.0000	1.0000
MulticlassF1Score	0.0000	0.0000	0.0000	0.0672
MulticlassF1Score (micro average):	0.0348
